In [ ]:
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
import multiprocessing as mp
import os
import math

In [ ]:
def cal_flow(G, T, sink_nodes, display):
    vertices = G.nodes()
    print("Vertices - ", vertices)

    inflow = dict(G.nodes().data("weight"))
    print("Inflow - ", inflow)

    edges = G.edges()
    print("Edges - ", edges)

    capacity={}
    cost={}
    for i,j in edges:
        d = G.edges[i,j]
        capacity[i,j] = d['capacity']
        cost[i,j] = d['length']

    print("Capacity - ", capacity)
    print("Cost - ", cost)
    
    if T < cost[max(cost, key=cost.get)]:
        return 0

    # create model
    m = gp.Model("Maximum Flow Problem")

    # define variables
    flow=gp.tupledict({})

    for i, j in edges:
        flow.update(m.addVars(i, j, range(cost[i,j],T+1), ub=capacity[i,j], name = "flow"))

    holdover_var = m.addVars(vertices, range(1,T+1), name = "holdover_var")

    dummy=gp.tupledict({})
    for i in vertices:
        dummy.update(m.addVars(i, lb=0, ub=inflow[i], name = "dummy"))

    # flow conservation 
    for i in vertices:
        for t in range(1,T):
            exp = holdover_var[i, t] + flow.sum('*', i, t) - holdover_var[i, t+1]
            for a,b in edges:
                if a==i:
                    exp -= flow.sum(a, b, t + cost[a , b])
            m.addConstr(exp==0,"node")

    # special case when t=0
    for i in vertices:
        expr = dummy[i] - holdover_var[i, 1]
        for x,y in edges:
            if x==i:
                expr += flow[x, y, cost[x,y]]
        m.addConstr(expr==0)

    #special case when t=T
    m.addConstrs(flow.sum('*', j , T) + holdover_var[j, T] == 0 for j in vertices if j not in sink_nodes)

    # objective maximize
    obj=dummy.sum()
    m.setObjective(obj, GRB.MAXIMIZE)
    
    print("TIME - ", T)
    m.optimize()
    model = m
    if m.ObjVal == sum(inflow.values()):
        if display == 1:
            for v in m.getVars():
                print(v.VarName, v.X)
            print('Time:', T)
            print('Obj: ',m.ObjVal)
        return 1
    return 0


def T_Max(graph):
    #nx.diameter(graph)
    diameter = max([max(j.values()) for (i,j) in nx.shortest_path_length(graph, None, None, 'length')])

    inflow = dict(graph.nodes().data("weight"))
    total_weight = sum(inflow.values())

    edges = graph.edges()
    capacity = []
    for i,j in edges:
            d = graph.edges[i,j]
            capacity.append(d['capacity'])
    min_capacity = min(capacity)
    
    return ((total_weight / min_capacity) + diameter)


In [ ]:
def main():
    graph = nx.read_gml("basicgraph.gml")
    #time = 12
    sink_nodes = ['T']
    #cal_flow(graph, time, sink_nodes)

    t_max = T_Max(graph)
    #upper_bound = t_max
    t_min = 0
    del_t = t_max - t_min

    #p = 4
    ncpus = int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
    pool = mp.Pool(processes=ncpus)

    flag = 0
    while(del_t > ncpus):
        time_p = []
        for i in range(0,ncpus):
            time_p.append(((i+1) * (math.ceil(((t_max - t_min)/(ncpus+1))))) + t_min)

        print(time_p)

        results = [pool.apply_async(cal_flow, args=(graph, x, sink_nodes, 0)) for x in time_p]
        f_vals = [p.get() for p in results]

        length = len(f_vals)
        flag = 0
        for i in range(0, length-1):
            if f_vals[i] == 0 and f_vals[i+1] == 1:
                t_min = time_p[i]
                t_max = time_p[i+1]
                flag = 1
                break

            elif f_vals[i] == 1 and f_vals[i+1] == 1:
                #t_min = 0           #lowerbound
                t_max = time_p[i]
               # upper_bound = t_max
                flag = 1
                break


        if flag == 0:
            t_min = time_p[ncpus-1]
            break

        del_t = t_max - t_min


    #upper_bound = t_max
    ncpus = del_t
    pool = mp.Pool(processes=ncpus)
    time_p = []
    for i in range(0,ncpus):
        time_p.append(((i+1) * (math.ceil(((t_max - t_min)/(ncpus+1))))) + t_min)

    print(time_p)

    results = [pool.apply_async(cal_flow, args=(graph, x, sink_nodes, 0)) for x in time_p]
    f_vals = [p.get() for p in results]

    length = len(f_vals)
    #flag = 0
    for i in range(0, length-1):
        if f_vals[i] == 1:
            t_max = time_p[i]
            #flag = 1
            break

    #if(flag == 0):
        #t_max = upper_bound

    cal_flow(graph, t_max, sink_nodes, 1)
    return 0  

main()